# **Cloning and Building Darknet**

In this part, we do the following tasks:

*   cloning darknet repo
*   changing makefile to have GPU and OPENCV enabled
*   build darknet

After the above tasks, we have config file for YOLO in the **cfg/** subdirectory.

In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15054, done.
remote: Total 15054 (delta 0), reused 0 (delta 0), pack-reused 15054
Receiving objects: 100% (15054/15054), 13.40 MiB | 13.72 MiB/s, done.
Resolving deltas: 100% (10233/10233), done.


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
# build darknet
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:935:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1136:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

# **Training YOLO on VOC**

In this part, we train YOLO with PASCAL VOC dataset. The milestones of this part are listed in below:

*   Get the PASCAL VOC data
*   Generate labels for VOC
*   Configure parameters in **cfg/yolov3-voc.cfg** and **cfg/voc.data**
*   Download pretrained convolutional weights
*   Train the model


## Get the PASCAL VOC data

Here we get all of the VOC data from 2007 to 2012. Then, we exctract the **.tar** files, so that we have **VOCdevkit/** subdirectory with all the VOC training data in it. 

In [ ]:
%cd ..

/content


In [ ]:
!wget https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar
!wget https://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
!wget https://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar

--2021-06-01 14:50:02--  https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1999639040 (1.9G) [application/octet-stream]
Saving to: ‘VOCtrainval_11-May-2012.tar’

VOCtrainval_11-May- 100%[===================>]   1.86G  22.6MB/s    in 83s     

2021-06-01 14:51:26 (22.9 MB/s) - ‘VOCtrainval_11-May-2012.tar’ saved [1999639040/1999639040]

--2021-06-01 14:51:26--  https://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 460032000 (439M) [application/octet-stream]
Saving to: ‘VOCtrainval_06-Nov-2007.tar’

VOCtrainval_06-Nov- 100%[===================>] 438.72M  23.1MB/s    in 20s     

2021-06-01 14:51:47 (22.1 MB

In [ ]:
!tar xf VOCtrainval_11-May-2012.tar
!tar xf VOCtrainval_06-Nov-2007.tar
!tar xf VOCtest_06-Nov-2007.tar

## Generate labels for VOC

Then we generate the label files that Darknet uses. Darknet wants a **.txt** file for each image with a line for each ground truth object in the image that looks like: 

`<object-class> <x> <y> <width> <height>**`

where x, y, width, and height are relative to the image's width and height. To generate these file we run the **voc_label.py** script.


In [ ]:
!wget https://pjreddie.com/media/files/voc_label.py

--2021-06-01 14:53:06--  https://pjreddie.com/media/files/voc_label.py
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2042 (2.0K) [application/octet-stream]
Saving to: ‘voc_label.py’

voc_label.py        100%[===================>]   1.99K  --.-KB/s    in 0s      

2021-06-01 14:53:06 (335 MB/s) - ‘voc_label.py’ saved [2042/2042]



In [ ]:
%run voc_label.py

<Figure size 432x288 with 0 Axes>

It generates a lot of label files in **VOCdevkit/VOC2007/labels/** and **VOCdevkit/VOC2012/labels/**. In your directory you should see:

```
ls
2007_test.txt   VOCdevkit
2007_train.txt  voc_label.py
2007_val.txt    VOCtest_06-Nov-2007.tar
2012_train.txt  VOCtrainval_06-Nov-2007.tar
2012_val.txt    VOCtrainval_11-May-2012.tar
```



 Darknet needs one text file with all of the images you want to train on. In this example, we train with everything except the 2007 test set so that we can test our model.

In [ ]:
%cat 2007_train.txt 2007_val.txt 2012_*.txt > train.txt

Now we have all the 2007 trainval and the 2012 trainval set in one big list. That's all we have to do for data setup!

## Configure parameters


In this part, we need to know and configure 3 files:


*   **data/voc.names**
*   **cfg/voc.data**
*   **cfg/yolov3-voc.cfg**


**data/voc.names** includes all classes that can be predicted by YOLO. Originally it looks like:


```
aeroplane
bicycle
bird
boat
bottle
bus
car
cat
chair
cow
diningtable
dog
horse
motorbike
person
pottedplant
sheep
sofa
train
tvmonitor
```



 In oder to point our data, we need to edit **cfg/voc.data**. Originally it looks like:


```
classes= 20
train  = /home/pjreddie/data/voc/train.txt
valid  = /home/pjreddie/data/voc/2007_test.txt
names = data/voc.names
backup = /home/pjreddie/backup/
```
We edited this file like below:


In [ ]:

%cat /content/darknet/cfg/voc.data

```
classes= 20
train  = /content/train.txt
valid  = /content/2007_test.txt
names = data/voc.names
backup = backup
```



Finally, we defined our parameters in **yolov3-voc.cfg**


*   batch = 64
*   subdivisions = 16
*   max_batches = 4000


In [ ]:
%cat /content/darknet/cfg/yolov3-voc.cfg

[net]
# Testing
 batch=64
 subdivisions=16
# Training
# batch=64
# subdivisions=16
width=416
height=416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 4000
policy=steps
steps=40000,45000
scales=.1,.1



[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

# Downsample

[convolutional]
batch_normalize=1
filters=64
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=32
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[shortcut]
from=-3
activation=linear

# Downsample

[convolutional]
batch_normalize=1
filters=128
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[shortcut]
from=-3
activation=linear

[convolutio

## Download pretrained convolutional weights

For training we use convolutional weights that are pre-trained on Imagenet. We use weights from the darknet53 model.

In [ ]:
%pwd

'/content'

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-06-01 14:56:50--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  23.1MB/s    in 7.4s    

2021-06-01 14:56:58 (20.9 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



## Train the model

In [ ]:
%cd darknet

/content/darknet


In [ ]:
!./darknet detector train cfg/voc.data cfg/yolov3-voc.cfg darknet53.conv.74 -dont_show 

Streaming output truncated to the last 5000 lines.
 total_bbox = 576243, rewritten_bbox = 0.384907 % 

 3926: 1.047828, 0.952123 avg loss, 0.001000 rate, 5.679109 seconds, 251264 images, 0.178767 hours left
Loaded: 0.000082 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.826066), count: 6, class_loss = 1.086227, iou_loss = 0.463599, total_loss = 1.549826 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.134712, iou_loss = 0.000000, total_loss = 0.134712 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000005, iou_loss = 0.000000, total_loss = 0.000005 
 total_bbox = 576249, rewritten_bbox = 0.384903 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.701868), count: 9, class_loss = 1.610118, iou_loss = 1.056588, total_loss = 2.666706 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1

We zip **.weigths ** files created by the training. We will use them for testing. 

In [ ]:
!zip -r /content/results.zip /content/darknet/backup/

  adding: content/darknet/backup/ (stored 0%)
  adding: content/darknet/backup/yolov3-voc_final.weights (deflated 7%)
  adding: content/darknet/backup/yolov3-voc_last.weights (deflated 7%)
  adding: content/darknet/backup/yolov3-voc_3000.weights (deflated 7%)
  adding: content/darknet/backup/yolov3-voc_2000.weights (deflated 7%)
  adding: content/darknet/backup/yolov3-voc_1000.weights (deflated 7%)
  adding: content/darknet/backup/yolov3-voc_4000.weights (deflated 7%)
